In [1]:
from collections import defaultdict
from itertools import product
import time

In [2]:
f = open('day17_input.txt')
lines = f.readlines()
f.close()
data = [(line[0:-1]) for line in lines]

In [3]:
print('Data:')
data

Data:


['####...#',
 '......##',
 '####..##',
 '##......',
 '..##.##.',
 '#.##...#',
 '....##.#',
 '.##.#.#.']

# Part 1

This is another questions similar to Conway's Game of Life (https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life#).

Because it's in 3D, there are **26** neighbors. "#" means active, and "." means inactive.

The rules are:
- If a cube is active and exactly 2 or 3 of its neighbors are also active, the cube remains active. Otherwise, the cube becomes inactive.
- If a cube is inactive but exactly 3 of its neighbors are active, the cube becomes active. Otherwise, the cube remains inactive.

First, I'll define a function which counts the status of the 26 neighbors:

In [4]:
def count_neighbors(xyz: str, cube) -> int:
    """
    args:
        xyz: the 3d coordinates (x, y, z) in string format "x,y,z"
        cube: the previous state of the 3D cube
    returns:
        number of active neighbors out of the 26 neighbors.
    """
    x, y, z = [int(n) for n in xyz.split(',')]
    ct = -cube[xyz] if xyz in cube else 0
    neighbors = []
    for i, j, k in product([-1, 0, 1], [-1, 0, 1], [-1, 0, 1]):
        neighbor = f'{x+i},{y+j},{z+k}'
        if neighbor in cube:
            ct += cube[neighbor]
    return ct

The initial state is:

In [5]:
cube0 = defaultdict(lambda:0)
N = len(data)
for x in range(N):
    for y in range(N):
        xyz = f'{x - N//2},{y - N//2},0'   # I prefer to put the (0, 0, 0) at the center
        if data[x][y] == '#':
            cube0[xyz] += 1
cube0

defaultdict(<function __main__.<lambda>()>,
            {'-4,-4,0': 1,
             '-4,-3,0': 1,
             '-4,-2,0': 1,
             '-4,-1,0': 1,
             '-4,3,0': 1,
             '-3,2,0': 1,
             '-3,3,0': 1,
             '-2,-4,0': 1,
             '-2,-3,0': 1,
             '-2,-2,0': 1,
             '-2,-1,0': 1,
             '-2,2,0': 1,
             '-2,3,0': 1,
             '-1,-4,0': 1,
             '-1,-3,0': 1,
             '0,-2,0': 1,
             '0,-1,0': 1,
             '0,1,0': 1,
             '0,2,0': 1,
             '1,-4,0': 1,
             '1,-2,0': 1,
             '1,-1,0': 1,
             '1,3,0': 1,
             '2,0,0': 1,
             '2,1,0': 1,
             '2,3,0': 1,
             '3,-3,0': 1,
             '3,-2,0': 1,
             '3,0,0': 1,
             '3,2,0': 1})

In the "cube" dictionary, for example, "'1,-4,0': 1," means the grid at x = 1, y = -4, z = 0 is active.

In [6]:
cycle = 0
cube = dict(cube0)
t0 = time.time()
while cycle < 6:
    cycle += 1

    # expand the x, y, and z range by 2 (+/-1) each cycle
    Nx = N + 2*cycle    
    Nz = 1 + 2*cycle
    
    # we should not modify the previous state, so a new cube is created:
    new_cube = dict(cube)
    
    for i, j, k in product(range(Nx), range(Nx), range(Nz)):
        xyz = f'{i - Nx//2},{j- Nx//2},{k - Nz//2}'
        ct = count_neighbors(xyz, cube)

        if (xyz in cube) and (cube[xyz] == 1):
            if (ct == 2) or (ct == 3):
                new_cube[xyz] = 1
            else:
                new_cube[xyz] = 0
        else:
            if ct == 3:
                new_cube[xyz] = 1
            else:
                new_cube[xyz] = 0
    cube = new_cube

print(f'There are {sum(new_cube.values())} active cubes in the active state after {cycle} cycles.')
print(f'This step cost {round(time.time() - t0, 2)} sec.')

There are 237 active cubes in the active state after 6 cycles.
This step cost 0.46 sec.


# Part 2

3D -> 4D

We need to modify the count_neighbor funciton so that it counts 4D:

In [7]:
def count_neighbors4D(xyzw: str, cube) -> int:
    """
    args:
        xyzw: the 4d coordinates (x, y, z) in string format "x,y,z,w"
        cube: the previous state of the 4D cube
    returns:
        number of active neighbors out of the 26 neighbors.
    """
    x, y, z, w = [int(n) for n in xyzw.split(',')]
    ct = -cube[xyzw] if xyzw in cube else 0
    neighbors = []
    for i, j, k, l in product([-1, 0, 1], [-1, 0, 1], [-1, 0, 1], [-1, 0, 1]):
        neighbor = f'{x+i},{y+j},{z+k},{w+l}'
        if neighbor in cube:
            ct += cube[neighbor]
    return ct

And do the same for the rest:

In [8]:
cube0 = defaultdict(lambda:0)
N = len(data)
for x in range(N):
    for y in range(N):
        xyzw = f'{x - N//2},{y - N//2},0,0'   # I prefer to put the (0, 0, 0, 0) at the center
        if data[x][y] == '#':
            cube0[xyzw] += 1

In [9]:
cycle = 0
cube = dict(cube0)
t0 = time.time()
while cycle < 6:
    cycle += 1

    # expand the x, y, and z range by 2 (+/-1) each cycle
    Nx = N + 2*cycle    
    Nz = 1 + 2*cycle
    
    # we should not modify the previous state, so a new cube is created:
    new_cube = dict(cube)
    
    for i, j, k, w in product(range(Nx), range(Nx), range(Nz), range(Nz)):
        xyzw = f'{i - Nx//2},{j- Nx//2},{k - Nz//2},{w - Nz//2}'
        ct = count_neighbors4D(xyzw, cube)

        if (xyzw in cube) and (cube[xyzw] == 1):
            if (ct == 2) or (ct == 3):
                new_cube[xyzw] = 1
            else:
                new_cube[xyzw] = 0
        else:
            if ct == 3:
                new_cube[xyzw] = 1
            else:
                new_cube[xyzw] = 0
    cube = new_cube

print(f'There are {sum(new_cube.values())} active cubes in the active state after {cycle} cycles.')
print(f'This step cost {round(time.time() - t0, 2)} sec.')

There are 2448 active cubes in the active state after 6 cycles.
This step cost 12.65 sec.
